In [3]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import scipy.special

In [4]:
N = 8
K = 4

In [5]:
def full_adder(a,b,c):
    s = (a ^ b) ^ c
    c = (a & b) | (c & (a ^ b))
    return s,c

def add_bool(a,b):
    k = len(a)
    s = np.zeros(k,dtype=bool)
    c = False
    for i in reversed(range(0,k)):
        s[i], c = full_adder(a[i],b[i],c)    
    
    return s

def inc_bin(a):
    k = len(a)
    increment = np.hstack((np.zeros(k-1,dtype=int), np.ones(1,dtype=int)))
    a = add_bool(a,increment)
    return a

def approx(C):
    W = inv_fc(1-(1-pi_fc(C))**2)  
    return W

def pi_fc(x):
    a = -0.4527
    b = 0.0218
    r = 0.86
    
    if x < 10:
        P = np.exp(a*(x**r)+b)
    else:
        P = np.sqrt((np.pi)/x)*np.exp(-x/4)*(1-10/(7*x))
        
    return P

def inv_fc(x):
    a = -0.4527
    b = 0.0218
    r = 0.86
    
    I = ((np.log(x)-b)/a)**(1/r)
    
    return I
    

def polarization(N, K, sigma):
    
    W = np.ones(N, dtype=float)
    W_temp = np.zeros(N, dtype=float)
    
    W[0] = (2/(sigma**2))
    
    for i in range(1, int(np.log2(N))+1):
        W_temp[:] = W[:]
        t = 2**(i-1)
        
        for j in range(1,(int(t))+1):
            C = W_temp[j-1]
            W[2*j-1-1] = approx(C)         #polarization by using density evolution
            W[2*j-1] = 2*C
                  
    qfunc = lambda x: 0.5-0.5*scipy.special.erf(x/np.sqrt(2))
    W = qfunc(np.sqrt(W/2))
    idx = sorted(range(W.size), key=lambda k: -W[k]) # descend order 
    A = idx[-K:]

    
    return A
    
def encoding(u, N):

    n = 1
    x = np.copy(u)
    step = np.log2(N)
    for s in range(0,step.astype(int)):
        i = 0
        while i < N:
            for j in range(0,n):
                idx = i+j
                x[idx] = x[idx] ^ x[idx+n]
            i=i+2*n
        n=2*n
        
    for s in range(0,N):
        if np.mod(x[s],2) == 0:
            x[s] = 0

    return x

In [6]:
snr_db = 1000
sigma = np.sqrt(1/(2*10**(snr_db/10)))
print(sigma)

7.07106781187e-51


In [7]:
b = np.zeros((2**K,K),dtype=int)
for i in range(1,2**K):
    b[i]= inc_bin(b[i-1])
    
b = (-1)**b

In [8]:
A = polarization(N, K, sigma)
u = np.zeros((2**K, N), dtype=int)
u[:,A] = b

/usr/local/lib/python3.6/site-packages/ipykernel/__main__.py:42: RuntimeWarning: divide by zero encountered in log


In [9]:
A

[4, 5, 6, 7]

In [10]:
x = np.zeros((2**K, N), dtype=int)

for i in range(0, 2**K):
    x[i,:] = encoding(u[i,:], N)

In [11]:
b[0:5,:]

array([[ 1,  1,  1,  1],
       [ 1,  1,  1, -1],
       [ 1,  1, -1,  1],
       [ 1,  1, -1, -1],
       [ 1, -1,  1,  1]])

In [12]:
u[0:5,:]

array([[ 0,  0,  0,  0,  1,  1,  1,  1],
       [ 0,  0,  0,  0,  1,  1,  1, -1],
       [ 0,  0,  0,  0,  1,  1, -1,  1],
       [ 0,  0,  0,  0,  1,  1, -1, -1],
       [ 0,  0,  0,  0,  1, -1,  1,  1]])

In [13]:
x[0:5,:]

array([[ 0,  0,  0,  1,  0,  0,  0,  1],
       [ 0,  0,  0, -1,  0,  0,  0, -1],
       [ 0,  0,  0,  1,  0,  0,  0,  1],
       [ 0,  0,  0, -1,  0,  0,  0, -1],
       [ 0,  0,  0,  1,  0,  0,  0,  1]])

In [76]:
Layer1 = 128
Layer2 = 64
Layer3 = 32
Layer4 = N


X = tf.placeholder(tf.float32,[None, N])
Y = tf.placeholder(tf.float32,[None, N])

W1 = tf.Variable(tf.random_uniform([N, Layer1], -1.0, 1.0))
W2 = tf.Variable(tf.random_uniform([Layer1, Layer2], -1.0, 1.0))
W3 = tf.Variable(tf.random_uniform([Layer2, Layer3], -1.0, 1.0))
W4 = tf.Variable(tf.random_uniform([Layer3, N], -1.0, 1.0))

b1 = tf.Variable(tf.zeros([Layer1]), name = "Bias1")
b2 = tf.Variable(tf.zeros([Layer2]), name = "Bias2")
b3 = tf.Variable(tf.zeros([Layer3]), name = "Bias3")
b4 = tf.Variable(tf.zeros([Layer4]), name = "Bias4")

L1 = tf.nn.sigmoid(tf.matmul(X, W1) + b1)
L2 = tf.nn.sigmoid(tf.matmul(L1, W2) + b2)
L3 = tf.nn.sigmoid(tf.matmul(L2, W3) + b3)
hypo = tf.nn.tanh(tf.matmul(L3, W4) + b4)

cost = -tf.reduce_mean(tf.maximum(Y,0)*tf.log(hypo)+tf.maximum(-Y,0)*tf.log(-hypo)+((1-tf.abs(Y))*tf.log(1-hypo)))
cost = -tf.reduce_mean(tf.maximum(Y,0)*tf.log(hypo)+tf.maximum(-Y,0)*tf.log(-hypo)+((1-tf.abs(Y))*tf.log(1-hypo)))
#cost = -tf.reduce_mean(Y*tf.log(hypo)+(1-Y)*tf.log(1-hypo))
#cost = tf.reduce_mean(tf.square(Y-hypo))

a = 0.005
optimizer = tf.train.GradientDescentOptimizer(a)
train = optimizer.minimize(cost)

init = tf.global_variables_initializer()

In [77]:
sess = tf.Session()
sess.run(init)

for step in range(2**13):
    sess.run(train, feed_dict={X:x+np.random.normal(0,sigma,(2**K, N)), Y:u})
    
    if step % 1000 == 0:
        print(step)
        print(sess.run(hypo, feed_dict={X:x+np.random.normal(0,sigma,(2**K, N)), Y:u})) 
        #print(sess.run(W1, feed_dict={X:x, Y:u}))
        
correct_prediction = tf.equal(tf.round(2*hypo-1), Y)
#correct_prediction = tf.equal(tf.round(hypo), Y)

accuracy = tf.reduce_mean(tf.cast(correct_prediction,"float"))
print("Accuracy:")
print(sess.run(accuracy, feed_dict={X:x+np.random.normal(0,sigma,(2**K, N)), Y:u}))

0
[[-0.35163602 -0.99999988  0.90191936  0.45741862  0.90651929  0.99892312
   0.99964082 -0.99732029]
 [ 0.20475936  0.99999958 -0.93167561 -0.88668787 -0.75732231 -0.99986196
  -0.9998135   0.99596477]
 [-0.35163602 -0.99999988  0.90191936  0.45741862  0.90651929  0.99892312
   0.99964082 -0.99732029]
 [ 0.20475936  0.99999958 -0.93167561 -0.88668787 -0.75732231 -0.99986196
  -0.9998135   0.99596477]
 [-0.35163602 -0.99999988  0.90191936  0.45741862  0.90651929  0.99892312
   0.99964082 -0.99732029]
 [ 0.20475936  0.99999958 -0.93167561 -0.88668787 -0.75732231 -0.99986196
  -0.9998135   0.99596477]
 [-0.35163602 -0.99999988  0.90191936  0.45741862  0.90651929  0.99892312
   0.99964082 -0.99732029]
 [ 0.20475936  0.99999958 -0.93167561 -0.88668787 -0.75732231 -0.99986196
  -0.9998135   0.99596477]
 [-0.35163602 -0.99999988  0.90191936  0.45741862  0.90651929  0.99892312
   0.99964082 -0.99732029]
 [ 0.20475936  0.99999958 -0.93167561 -0.88668787 -0.75732231 -0.99986196
  -0.9998135   

In [64]:
u

array([[ 0,  0,  0,  0,  1,  1,  1,  1],
       [ 0,  0,  0,  0,  1,  1,  1, -1],
       [ 0,  0,  0,  0,  1,  1, -1,  1],
       [ 0,  0,  0,  0,  1,  1, -1, -1],
       [ 0,  0,  0,  0,  1, -1,  1,  1],
       [ 0,  0,  0,  0,  1, -1,  1, -1],
       [ 0,  0,  0,  0,  1, -1, -1,  1],
       [ 0,  0,  0,  0,  1, -1, -1, -1],
       [ 0,  0,  0,  0, -1,  1,  1,  1],
       [ 0,  0,  0,  0, -1,  1,  1, -1],
       [ 0,  0,  0,  0, -1,  1, -1,  1],
       [ 0,  0,  0,  0, -1,  1, -1, -1],
       [ 0,  0,  0,  0, -1, -1,  1,  1],
       [ 0,  0,  0,  0, -1, -1,  1, -1],
       [ 0,  0,  0,  0, -1, -1, -1,  1],
       [ 0,  0,  0,  0, -1, -1, -1, -1]])